In [689]:
import os
import cv2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

In [690]:
SRC = '/Users/fathe/OneDrive/Documents/UK/MFF/Thesis/input'
videos = [file for file in os.listdir(SRC) if file.endswith("MP4") and file.startswith("GX")]
SCALE = 4

In [691]:
videos

In [692]:
video = videos[-2]

In [693]:
vidcap = cv2.VideoCapture(os.path.join(SRC, video))

In [694]:
intervals_vertical = np.load(os.path.join(SRC, video).replace(".MP4", "-intervals_vertical.npy"))
intervals_horizontal = np.load(os.path.join(SRC, video).replace(".MP4", "-intervals_horizontal.npy"))

In [695]:
interval = intervals_horizontal[1]

In [696]:
interval

In [697]:
start = 10070
stop = 10072

In [698]:
img_range = range(stop-start)

In [699]:
len(img_range)

In [700]:
stop-start

In [701]:
def is_blurry(frame, threshold=100):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    return laplacian_var < threshold

In [702]:
feature_params = dict(maxCorners=100,
                          qualityLevel=0.3,
                          minDistance=7,
                          blockSize=7)
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
                 # minEigThreshold = 0.045)
                 
err_threshold = 8

In [703]:
plt.figure(figsize=(16,12*len(img_range)))
# vidcap.set(cv2.CAP_PROP_POS_FRAMES, start)
# pbar = tqdm(total=stop-start)
# blurs = []
for i in img_range:
    vidcap.set(cv2.CAP_PROP_POS_FRAMES, start+i)
    _, frame0 = vidcap.read()
    _, frame1 = vidcap.read()
    frame0 = cv2.cvtColor(cv2.resize(frame0, (int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)/SCALE), int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)/SCALE))), cv2.COLOR_BGR2GRAY)
    frame1 = cv2.cvtColor(cv2.resize(frame1, (int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)/SCALE), int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)/SCALE))), cv2.COLOR_BGR2GRAY)
    
    p0 = cv2.goodFeaturesToTrack(frame0, **feature_params)
    p1, st, err = cv2.calcOpticalFlowPyrLK(frame0, frame1, p0, None, **lk_params)
    # print(err)
    # st = (st == 1) & (err < err_threshold)
    p1 = p1[st == 1]
    p0 = p0[st == 1]
    print(cv2.Laplacian(frame0, cv2.CV_64F).var())
    print(start+i, np.mean(p1-p0, axis=0))
    print(cv2.Laplacian(frame1, cv2.CV_64F).var())

    for p in p0:
        x, y = p
        cv2.circle(frame0, (int(x), int(y)), int(20/SCALE), (0, 255, 0), int(3/SCALE))

    for p in p1:
        x, y = p
        cv2.circle(frame1, (int(x), int(y)), int(20/SCALE), (0, 255, 0), int(3/SCALE))
        
    # blurs.append(cv2.Laplacian(frame0, cv2.CV_64F).var())
    # pbar.update(1)
    

    ax = plt.subplot((stop-start),2,2*i+1)
    ax.imshow(frame0, cmap='gray')
    ax.set_title(i+start)

    ax = plt.subplot((stop-start),2,2*i+2)
    ax.imshow(frame1, cmap='gray')
    ax.set_title(i+start+1)
plt.show()

In [704]:
# np.mean(blurs), np.std(blurs)

In [705]:
# plt.figure(figsize=(15,5))
# plt.plot(img_range, blurs)
# plt.show()